In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 62.9 MB/s eta 0:00:00:00:0100:01


In [4]:
import torch
from utils import check_set_gpu
from data_utils import load_and_preprocess_data, prepare_data_splits, load_pretrained_embeddings
from models import CNNModel, BiLSTMModel, CNNLSTMModel, TransformerModel, PretrainedEmbeddingsModel, CNNRNNModel
from pipeline import ModelPipeline
import os

number_of_epochs = 100
    # Set device
device = check_set_gpu()

data, token_to_id, categorical_vectorizer, categorical_columns, UNK_IX, PAD_IX  = load_and_preprocess_data('./Train_rev1.csv')
data_train, data_val = prepare_data_splits(data)


Using GPU: NVIDIA RTX A6000
Splitting data...
Train size = 195814
Validation size = 48954


In [5]:
embedding_matrix = load_pretrained_embeddings(token_to_id, embedding_name='glove-wiki-gigaword-300')


[===-----------------------------------------------] 6.2% 23.4/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=======-------------------------------------------] 15.3% 57.6/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==============------------------------------------] 28.3% 106.2/376.1MB downloaded

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[=====================================-------------] 75.8% 285.0/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[===============================================---] 94.3% 354.8/376.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
from models import CNNBiLSTMEmbeddingModel

models_to_test = [    
    ('CNNBiLSTMEmbeddingModel1e-3', CNNBiLSTMEmbeddingModel, {
        'embedding_matrix': embedding_matrix,
        'n_cat_features': len(categorical_vectorizer.vocabulary_),  
        'lstm_hidden_size': [128],  
        'num_lstm_layers': [1],  #
        'dropout': [0.3],  
        'padding_idx': PAD_IX,  #
        'freeze': False, 
        'batch_size': [128],  
        'epochs': [100],  
    })
]

pipeline = ModelPipeline(
    models=models_to_test,
    data_train=data_train,
    data_val=data_val,
    token_to_id=token_to_id,
    categorical_vectorizer=categorical_vectorizer,
    categorical_columns=categorical_columns,
    UNK_IX=UNK_IX,
    PAD_IX=PAD_IX,
    log_dir='logs',
    checkpoint_dir='checkpoints',
    early_stopping_patience=10,
    device=device,
    epochs=200, 
    lr=1e-3
)

pipeline.train_and_evaluate()

2024-10-10 20:17:55,829 - Starting training for model: CNNBiLSTMEmbeddingModel1e-3 with hyperparams: {'embedding_matrix': tensor([[ 0.6889,  0.2095, -0.7360,  ..., -0.3863,  0.1655, -0.2924],
        [ 0.5227,  0.2222,  0.9701,  ...,  1.4729,  0.2297, -0.2529],
        [ 0.0385, -0.0398,  0.0827,  ..., -0.3343,  0.0118,  0.0597],
        ...,
        [ 0.4483,  1.2691, -0.1862,  ...,  0.1636,  0.2891, -0.6319],
        [-0.0924, -0.2186,  0.4416,  ...,  0.0976,  0.6912, -0.1121],
        [-0.2589,  0.3339,  0.0924,  ..., -1.0857, -1.0157, -0.2296]]), 'n_cat_features': 3768, 'padding_idx': 0, 'freeze': False, 'lstm_hidden_size': 128, 'num_lstm_layers': 1, 'dropout': 0.3, 'batch_size': 128, 'epochs': 100}
2024-10-10 20:17:55,870 - No checkpoint found for model CNNBiLSTMEmbeddingModel1e-3 with hyperparams {'embedding_matrix': tensor([[ 0.6889,  0.2095, -0.7360,  ..., -0.3863,  0.1655, -0.2924],
        [ 0.5227,  0.2222,  0.9701,  ...,  1.4729,  0.2297, -0.2529],
        [ 0.0385, -0.0398

Training CNNBiLSTMEmbeddingModel1e-3:   0%|          | 0/100 [00:00<?, ?it/s]

2024-10-10 20:19:56,177 - Epoch 1 - Validation MAE: 0.22262, Total Loss: 123561.43924
2024-10-10 20:21:57,156 - Epoch 2 - Validation MAE: 0.18920, Total Loss: 30490.72486
2024-10-10 20:23:58,081 - Epoch 3 - Validation MAE: 0.19244, Total Loss: 27760.10041
2024-10-10 20:25:58,985 - Epoch 4 - Validation MAE: 0.18098, Total Loss: 25797.01498
2024-10-10 20:27:59,187 - Epoch 5 - Validation MAE: 0.17743, Total Loss: 24415.07373
2024-10-10 20:29:59,263 - Epoch 6 - Validation MAE: 0.20234, Total Loss: 22909.84503
2024-10-10 20:31:58,850 - Epoch 7 - Validation MAE: 0.17686, Total Loss: 21503.87433
2024-10-10 20:33:58,563 - Epoch 8 - Validation MAE: 0.19580, Total Loss: 20610.81554
2024-10-10 20:35:57,717 - Epoch 9 - Validation MAE: 0.17108, Total Loss: 19598.50288
2024-10-10 20:37:57,632 - Epoch 10 - Validation MAE: 0.16818, Total Loss: 18773.01518
2024-10-10 20:39:57,169 - Epoch 11 - Validation MAE: 0.16934, Total Loss: 17722.30276
2024-10-10 20:41:57,255 - Epoch 12 - Validation MAE: 0.17531, 

                    model_name  \
0  CNNBiLSTMEmbeddingModel1e-3   

                                         hyperparams   val_mae  
0  {'embedding_matrix': [[tensor(0.6889), tensor(...  0.143732  
